# Query SAP table and join with local data

A first example demonstrates how to join two SAP tables with an external table. We’ll be using the [ABAP Flight Reference Scenario](https://help.sap.com/docs/ABAP_PLATFORM_NEW/fc4c71aa50014fd1b43721701471913d/a9d7c7c140a0408dbc5966c52d156b49.html), specifically joining the `SFLIGHT` and `SPFLI` tables which contain flight and flight schedule details respectively, with an external table `WEATHER` that holds weather information. We will extract flight information and associated temperatures at departure and arrival cities.

## Import DuckDB & load **ERPL** extension
In the next cells we import duckdb. Then we install the ERPL extension and load it into the current DB-session. Via multiple SET-commands we configure the connection to our SAP development system. In our case we use the docker based [ABAP Platform Trial](https://hub.docker.com/r/sapse/abap-platform-trial). The credentials are set by default, details can be found in the documentation of the docker image. 

In [1]:
var duckdb = require('duckdb');
var db = new duckdb.Database(':memory:', {"allow_unsigned_extensions": "true"});
db.all(`
LOAD 'erpl';

SET sap_ashost = 'localhost';
SET sap_sysnr = '00';
SET sap_user = 'DEVELOPER';
SET sap_password = 'Htods70334';
SET sap_client = '001';
SET sap_lang = 'EN';
`, (err, res) => console.log(err || res));

Database { default_connection: Connection {} }

[]


In [2]:
db.all(`PRAGMA sap_rfc_ping`, (err, res) => console.log(err || res[0]["msg"]));

Database { default_connection: Connection {} }

PONG


If the loading of the extension was successful, we can find the exportet functions in the list of `duckdb_functions()`

In [5]:
db.all(`SELECT * FROM duckdb_functions() WHERE function_name LIKE '%sap%'`, 
       (err, res) => console.log(err || res.map(el => el["function_name"])));

Database { default_connection: Connection {} }

[
  'sap_describe_fields',
  'sap_show_tables',
  'sap_rfc_describe_function',
  'sap_read_table',
  'sap_rfc_search_function',
  'sap_rfc_search_group',
  'sap_rfc_invoke',
  'sap_rfc_ping',
  'sap_rfc_function_desc',
  'sap_rfc_set_trace_level',
  'sap_rfc_set_trace_dir',
  'sap_rfc_set_maximum_trace_file_size',
  'sap_rfc_set_maximum_stored_trace_files',
  'sap_rfc_set_ini_path',
  'sap_rfc_reload_ini_file'
]


## Explore the schema of the relevant tables

The ERPL extension provides the method `sap_describe_fields` to explore the data dictionary schema of the respective table. For exploring local data we also can use the `DESCRIBE` command to get the fields of e.g. a CSV-file.

In [6]:
db.all(`SELECT * FROM sap_describe_fields('SFLIGHT');`, 
       (err, res) => console.log(err || res));

Database { default_connection: Connection {} }

[
  {
    pos: '0001',
    is_key: 'X',
    field: 'MANDT',
    text: 'Client',
    sap_type: 'CLNT',
    length: '000003',
    decimals: '000000',
    check_table: 'T000',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0002',
    is_key: 'X',
    field: 'CARRID',
    text: 'Airline Code',
    sap_type: 'CHAR',
    length: '000003',
    decimals: '000000',
    check_table: 'SCARR',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0003',
    is_key: 'X',
    field: 'CONNID',
    text: 'Flight Connection Number',
    sap_type: 'NUMC',
    length: '000004',
    decimals: '000000',
    check_table: 'SPFLI',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0004',
    is_key: 'X',
    field: 'FLDATE',
    text: 'Flight date',
    sap_type: 'DATS',
    length: '000008',
    decimals: '000000',
    check_table: '',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0005',
    is_key: '',
   

In [7]:
db.all(`SELECT * FROM sap_describe_fields('SPFLI');`, 
       (err, res) => console.log(err || res));

Database { default_connection: Connection {} }

[
  {
    pos: '0001',
    is_key: 'X',
    field: 'MANDT',
    text: 'Client',
    sap_type: 'CLNT',
    length: '000003',
    decimals: '000000',
    check_table: 'T000',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0002',
    is_key: 'X',
    field: 'CARRID',
    text: 'Airline Code',
    sap_type: 'CHAR',
    length: '000003',
    decimals: '000000',
    check_table: 'SCARR',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0003',
    is_key: 'X',
    field: 'CONNID',
    text: 'Flight Connection Number',
    sap_type: 'NUMC',
    length: '000004',
    decimals: '000000',
    check_table: '',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0004',
    is_key: '',
    field: 'COUNTRYFR',
    text: 'Country Key',
    sap_type: 'CHAR',
    length: '000003',
    decimals: '000000',
    check_table: 'SGEOCITY',
    ref_table: '',
    ref_field: '',
    language: 'E'
  },
  {
    pos: '0005',
    is_key: ''

## Join tables query

The actual SQL query joins the three tables and performs the following operations:
- Retrieves flight details from `SFLIGHT` using ERPL's `sap_read_table`, aliasing it as `f`.
- Again using ERPL's `sap_read_table` we join `SPFLI` (aliased as `s`) on `MANDT`, `CARRID`, and `CONNID` to get the flight's city of origin and destination.
- Incorporates two instances of an external weather data CSV file, `w_from` and `w_to`, matching on flight date and respective cities' country and name for departure and arrival.
- Rounds the temperature data to one decimal place for readability.
- Orders the results by `CARRIER_ID`, `CONNECTION_ID`, and `FLIGHT_DATE`.
- Limits the output to the first 25 rows for a concise view.

The output of this query will provide a comprehensive view of the flights, including their departure and arrival cities, and the corresponding temperatures, thus offering valuable insights for flight operations analysis.

In [10]:
db.all(`
SELECT 
  f.CARRID,
  f.CONNID,
  f.FLDATE,
  s.CITYFROM as CITY_FROM,
  ROUND(w_from.TEMPERATURE, 1) as TEMP_FROM,
  s.CITYTO as CITY_TO,
  ROUND(w_to.TEMPERATURE, 1) as TEMP_TO,
  FROM sap_read_table('SFLIGHT') AS f
  JOIN sap_read_table('SPFLI') AS s 
      ON (f.MANDT = s.MANDT AND f.CARRID = s.CARRID AND f.CONNID = s.CONNID)
  JOIN 'WEATHER.csv' AS w_from
      ON (f.FLDATE = w_from.FLDATE AND s.COUNTRYFR = w_from.COUNTRY AND s.CITYFROM = w_from.CITY)
  JOIN 'WEATHER.csv' AS w_to
      ON (f.FLDATE = w_to.FLDATE AND s.COUNTRYTO = w_to.COUNTRY AND s.CITYTO = w_to.CITY)
  ORDER BY 1, 2, 3
  LIMIT 3
`, 
(err, res) => console.log(err || res));

Database { default_connection: Connection {} }

[
  {
    CARRID: 'AA',
    CONNID: '0017',
    FLDATE: 2016-11-15T00:00:00.000Z,
    CITY_FROM: 'NEW YORK',
    TEMP_FROM: 18.4,
    CITY_TO: 'SAN FRANCISCO',
    TEMP_TO: 26.3
  },
  {
    CARRID: 'AA',
    CONNID: '0017',
    FLDATE: 2017-02-03T00:00:00.000Z,
    CITY_FROM: 'NEW YORK',
    TEMP_FROM: 20,
    CITY_TO: 'SAN FRANCISCO',
    TEMP_TO: 18.1
  },
  {
    CARRID: 'AA',
    CONNID: '0017',
    FLDATE: 2017-04-24T00:00:00.000Z,
    CITY_FROM: 'NEW YORK',
    TEMP_FROM: 23.5,
    CITY_TO: 'SAN FRANCISCO',
    TEMP_TO: 10.3
  }
]
